   ![Texte alternatif](image.png)

In [1]:
from colorthief import ColorThief

color_thief = ColorThief("image.png")
palette = color_thief.get_palette(color_count=4)

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

plotly_colors = [rgb_to_hex(c) for c in palette]

# **HEART RISK Data Visualization**

Gauthier Meffe, Noa Chaze, Thomas Dos, Lisa Escaron

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
df_train = pd.read_csv('data/train.csv')

## Description du jeu de données

Lors d'un appel téléphonique, 325 questions sont posées (une pour chaque colonne de notre jeu de données) à 225 000 personnes. 

La première information donnée (première colonne STATE) est l'état de résidence de la personne (état vers lequel l'appel est envoyé). On peut ainsi visualiser sur la carte des Etat-Unis la provenance des participants.

In [4]:
import plotly.express as px

# Dictionnaire de correspondance entre numéro FIPS et le code état à deux lettres
fips_to_state = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT',
    '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL',
    '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
    '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE',
    '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD',
    '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV',
    '55': 'WI', '56': 'WY'
}

# Nombre de personnes par état
state_counts = df_train['_STATE'].value_counts().reset_index()
state_counts.columns = ['_STATE', 'count']

# Convertir FIPS en string à 2 chiffres puis en code état à 2 lettres
state_counts['_STATE'] = state_counts['_STATE'].astype(int).astype(str).str.zfill(2)
state_counts['state_code'] = state_counts['_STATE'].map(fips_to_state)

# Retirer les lignes sans correspondance (ex: territoires hors USA)
state_counts = state_counts.dropna(subset=['state_code'])

custom_gradient = ["white", plotly_colors[0]]

fig = px.choropleth(
    state_counts,
    locations='state_code',
    locationmode="USA-states",
    color='count',
    scope="usa",
    color_continuous_scale=custom_gradient,
    title="Nombre de personnes appelées par état"
)
fig.show()

Ces appels sont passés pour évaluer les causes possibles de risque cardiaque chez une personne. Ainsi la dernière question posée concerne les antécédents de crise cardiaque. Sur les personnes appelées, la répartition des personnes ayant eu une crise cardiaque est la suivante :

In [5]:
import plotly.graph_objects as go

# Calcul du pourcentage pour chaque classe de TARGET
target_counts = df_train['TARGET'].value_counts(normalize=True)
labels = ['Arrêt cardiaque', 'Aucun arrêt cardiaque']
values = [target_counts[True]*100, target_counts[False]*100]


colors = [plotly_colors[0], plotly_colors[1]]

fig_target = go.Figure(data=[
    go.Bar(
        x=labels,
        y=values,
        marker_color=colors,
        text=[f"{v:.1f}%" for v in values],
        textposition='auto'
    )
])

fig_target.update_layout(
    title="Répartition des personnes ayant déjà eu un arrêt cardiaque sur le panel des personnes appelées",
    yaxis_title="Pourcentage (%)",
    xaxis_title="",
    plot_bgcolor='white'
)
fig_target.show()

Moins de 10% des personnes appelées ont un antécédent de crisque cardiaque. Cette répartition sera importante à prendre en compte lors de la modélisation : il faudra sûrement gonfler les données de personnes ayant eu une crise cardiaque pour équilibrer le jeu de données et avoir un modèle d'entraînement plus efficace. 

Il faut prendre en compte que cette valeur n'est pas représentative du pourcentage de personnes ayant des épisodes cardiaques car il n'est pas possible d'avoir au téléphone une personne ayant eu une crise cardiaque mortelle.

On peut ensuite visualiser sur la carte des Etats-Unis le nombre de personnes ayant eu une crise cardiaque par état de résidence : 

In [6]:
# Calculer le nombre de personnes ayant eu un arrêt cardiaque par état
state_target_counts = df_train[df_train['TARGET'] == True]['_STATE'].value_counts().reset_index()
state_target_counts.columns = ['_STATE', 'cardiac_arrest_count']

# Convertir FIPS en string à 2 chiffres puis en code état à 2 lettres
state_target_counts['_STATE'] = state_target_counts['_STATE'].astype(int).astype(str).str.zfill(2)
state_target_counts['state_code'] = state_target_counts['_STATE'].map(fips_to_state)

# Retirer les lignes sans correspondance (ex: territoires hors USA)
state_target_counts = state_target_counts.dropna(subset=['state_code'])


fig_cardiac = px.choropleth(
    state_target_counts,
    locations='state_code',
    locationmode="USA-states",
    color='cardiac_arrest_count',
    scope="usa",
    color_continuous_scale=custom_gradient,
    title="Nombre de personnes ayant eu un arrêt cardiaque par état"
)
fig_cardiac.show()

Cette carte n'a pas beaucoup de sens car on a vu précédemment qu'il n'y a pas eu le même nombre de personne intérrogées par état. On peut donc normaliser le nombre de personnes ayant eu une crise cardiaque par le nombre total de personnes appelées dans chaque état. Par la suite, nous travaillerons toujours avec des taux de crise cardiaque car le nombre de personnes appelées ne permet pas d'avoir le même échantillon pour chaque variable étudiée.

On obtient alors la carte suivante :

In [7]:
# Calculer le taux de crise cardiaque par état (nombre de cas / nombre d'appels)
merged = state_counts.merge(state_target_counts, on='state_code', how='left')
merged['cardiac_arrest_count'] = merged['cardiac_arrest_count'].fillna(0)
merged['cardiac_arrest_rate'] = merged['cardiac_arrest_count'] / merged['count']

fig_rate = px.choropleth(
    merged,
    locations='state_code',
    locationmode="USA-states",
    color='cardiac_arrest_rate',
    scope="usa",
    color_continuous_scale=custom_gradient,
    title="Taux de crise cardiaque par état (parmi les personnes appelées)"
)
fig_rate.show()

In [8]:
import plotly.graph_objects as go

# Trier les états par taux de crise cardiaque décroissant et prendre les 4 premiers
top4 = merged.sort_values('cardiac_arrest_rate', ascending=False).head(4).reset_index(drop=True)

# Dictionnaire code état → nom complet
state_names = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California",
    "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia",
    "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", "IN": "Indiana", "IA": "Iowa",
    "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi", "MO": "Missouri",
    "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio",
    "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont",
    "VA": "Virginia", "WA": "Washington", "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming"
}

top4['state_name'] = top4['state_code'].map(state_names)

size_factor = 1000
bubble_sizes = top4['cardiac_arrest_rate'] * size_factor

bubble_colors = plotly_colors[:4]

fig_bubble = go.Figure()
for i, row in top4.iterrows():
    fig_bubble.add_trace(go.Scatter(
        x=[i], y=[1],
        mode='markers+text',
        marker=dict(
            size=bubble_sizes.iloc[i],
            color=bubble_colors[i % len(bubble_colors)],
            line=dict(width=2, color='white')
        ),
        text=f"{row['state_name']}<br>{row['cardiac_arrest_rate']:.2%}",
        textposition='middle center',
        hoverinfo='skip',
        showlegend=False
    ))

fig_bubble.update_layout(
    title="Etats avec le taux de crise cardiaque le plus élevé",
    xaxis=dict(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
        range=[-0.5, 3.5]
    ),
    yaxis=dict(
        showticklabels=False,
        showgrid=False,
        zeroline=False,
        range=[0.5, 1.5]
    ),
    plot_bgcolor='white',
    height=350
)

fig_bubble.show()

On voit alors que certains états ont un taux de crise cardiaque bien plus élevés que d'autres, et ainsi que la première variable STATE est un indicateur à prendre en compte pour le modèle de prédiction, ce qui n'était pas forcément évident. 

Maintenant qu'on a vu géographiquement d'où venait les personnes appelées et d'où venait les personnes ayant eu déjà une crise cardiaque, on peut s'intéresser aux autres questions posées lors de l'appel téléphonique. Parmi les 325 questions posées, on en a sélectionné quelques unes qui nous semblaient intéressantes à analyser.

### **Profil des personnes appelées**

On peut visualiser l'âge et le sexe des personnes appelées. 

Toutes les personnes au dessus de 80 ans sont regroupées dans la même catégorie, on traite alors ce groupe à part pour avoir une meilleure visualisation de tous les âges en dessous. 

In [9]:
import plotly.express as px

# Supposons que 'SEXVAR' (1=Male, 2=Female) et '_AGEG5YR' (groupe d'âge) sont les colonnes d'intérêt
age_sex_counts = df_train.groupby(['_AGE80', 'SEXVAR']).size().reset_index(name='count')

# Mapping pour les labels de sexe
sex_labels = {1: 'Homme', 2: 'Femme'}
age_sex_counts['Sexe'] = age_sex_counts['SEXVAR'].map(sex_labels)

age_sex_counts_filtered = age_sex_counts[age_sex_counts['_AGE80'] < 80]
fig_age_sex = px.bar(
    age_sex_counts_filtered,
    x='_AGE80',
    y='count',
    color='Sexe',
    barmode='overlay',
    opacity=0.6,
    color_discrete_sequence=plotly_colors[:2],
    labels={'_AGE80': "Âges", 'count': 'Nombre de personnes'},
    title="Nombre de personnes appelées par groupe d'âge et par sexe (hors 80+)"
)
fig_age_sex.show()

In [10]:
# Filtrer les personnes de plus de 80 ans
age80_plus = df_train[df_train['_AGE80'] >= 80]

# Compter par sexe
age80_sex_counts = age80_plus['SEXVAR'].value_counts().reset_index()
age80_sex_counts.columns = ['SEXVAR', 'count']
age80_sex_counts['Sexe'] = age80_sex_counts['SEXVAR'].map(sex_labels)

import plotly.express as px

fig_age80_sex = px.bar(
    age80_sex_counts,
    x='Sexe',
    y='count',
    color='Sexe',
    color_discrete_sequence=plotly_colors[:2],
    title="Nombre de personnes de plus de 80 ans par sexe"
)
fig_age80_sex.show()

On a maintenant une idée du profil (âge et sexe) des personnes appelées. On a forcément plus de personnes au dessus de 80 ans car plus de tranches d'âges sont regroupées dans la même catégorie. Pour les personnes de moins de 80 ans, on remarque que la répartition est à peu près uniforme entre les différentes tranches d'âges, avec un peu plus de gens entre 60 et 80 ans. On peut penser que c'est une tranche d'âge qui a plus de temps pour répondre à ce genre d'appel.

On veut alors voir le nombre de crises cardiaques en fonction de l'âge et du sexe dans notre échantillon. Comme on a vu qu'on n'avait pas le même nombre de personnes dans chaque tranche d'âge, on va normaliser le nombre de crises cardiaques par le nombre total de personnes dans chaque tranche d'âge et dans chaque sexe.

In [11]:
# Nombre total de personnes appelées par âge et sexe (hors 80+)
age_sex_total = df_train[df_train['_AGE80'] < 80].groupby(['_AGE80', 'SEXVAR']).size().reset_index(name='total')
age_sex_total['Sexe'] = age_sex_total['SEXVAR'].map(sex_labels)

# Nombre de personnes ayant eu un arrêt cardiaque par âge et sexe (hors 80+)
cardiac_age_sex = df_train[(df_train['TARGET'] == True) & (df_train['_AGE80'] < 80)] \
    .groupby(['_AGE80', 'SEXVAR']).size().reset_index(name='cardiac_count')
cardiac_age_sex['Sexe'] = cardiac_age_sex['SEXVAR'].map(sex_labels)

# Fusionner pour calculer le taux
merged = age_sex_total.merge(cardiac_age_sex, on=['_AGE80', 'SEXVAR', 'Sexe'], how='left')
merged['cardiac_count'] = merged['cardiac_count'].fillna(0)
merged['taux_crise_cardiaque'] = merged['cardiac_count'] / merged['total']

import plotly.express as px

fig = px.bar(
    merged,
    x='_AGE80',
    y='taux_crise_cardiaque',
    color='Sexe',
    barmode='overlay',
    opacity=0.6,
    color_discrete_sequence=plotly_colors[:2],
    labels={'_AGE80': "Âges", 'taux_crise_cardiaque': 'Taux de crise cardiaque'},
    title="Taux de crise cardiaque par âge et par sexe (hors 80+)"
)
fig.show()

Les résultats sont très visuels : on remarque clairement que le taux de risque cardiaque augmente avec l'âge, ce qui est ce qu'on aurait pu prévoir. On remarque aussi que les hommes ont un taux de risque cardiaque plus élevé que les femmes dans toutes les tranches d'âges.Ce qui est aussi cohérent avec ce qu'on sait du risque cardiaque.

On peut maintenant s'intéresser à d'autres facteurs de risques cardiaques connues pour voir si on retrouve les mêmes tendances dans notre échantillon.


- Tabagisme :

In [12]:
# Calculer le taux de crise cardiaque pour chaque valeur de _SMOKER3
smoker3_group = df_train.groupby('_SMOKER3')
total_by_smoker3 = smoker3_group.size()
cardiac_by_smoker3 = smoker3_group['TARGET'].sum()
taux_cardiac_smoker3 = (cardiac_by_smoker3 / total_by_smoker3).reset_index()
taux_cardiac_smoker3.columns = ['_SMOKER3', 'Taux de crise cardiaque']

# Mapping des codes _SMOKER3 vers des labels compréhensibles
smoker3_labels = {
    1: 'Journalier',
    2: 'Occasionnel',
    3: 'Ancien fumeur',
    4: 'Jamais fumé'
}
taux_cardiac_smoker3['Statut fumeur'] = taux_cardiac_smoker3['_SMOKER3'].map(smoker3_labels)

import plotly.express as px

fig_smoker3 = px.bar(
    taux_cardiac_smoker3,
    x='Statut fumeur',
    y='Taux de crise cardiaque',
    color='Statut fumeur',
    color_discrete_sequence=plotly_colors,
    text=taux_cardiac_smoker3['Taux de crise cardiaque'].apply(lambda x: f"{x:.2%}"),
    title="Taux de crise cardiaque selon le statut tabagique"
)
fig_smoker3.update_traces(textposition='outside')
fig_smoker3.update_layout(showlegend=False, yaxis_tickformat='.1%')
fig_smoker3.show()

Ainsi, on retrouve bien au taux plus faible de crise cardiaque chez les non-fumeurs comparé aux fumeurs, ce qui est cohérent avec les connaissances médicales actuelles.

- Diabétisme :

In [13]:
# Mapping des valeurs de DIABETE4 vers des labels compréhensibles
diabete_labels = {
    1: "Diabétique",
    2: "Diabétique pendant la grossesse",
    3: "Non diabétique",
    4: "Pré-diabètique",
    7: "Inconnu",
    9: "Refusé"
}

# Filtrer les valeurs connues (1, 2, 3, 4)
df_diab = df_train[df_train['DIABETE4'].isin([1, 2, 3, 4])].copy()
df_diab['Diabète'] = df_diab['DIABETE4'].map(diabete_labels)

# Calcul du taux de crise cardiaque par statut diabète
diab_group = df_diab.groupby('Diabète')
total_by_diab = diab_group.size()
cardiac_by_diab = diab_group['TARGET'].sum()
taux_cardiac_diab = (cardiac_by_diab / total_by_diab).reset_index()
taux_cardiac_diab.columns = ['Diabète', 'Taux de crise cardiaque']

import plotly.express as px

fig_diab = px.bar(
    taux_cardiac_diab,
    x='Diabète',
    y='Taux de crise cardiaque',
    color='Diabète',
    color_discrete_sequence=plotly_colors,
    text=taux_cardiac_diab['Taux de crise cardiaque'].apply(lambda x: f"{x:.2%}"),
    title="Taux de crise cardiaque selon le statut diabétique"
)
fig_diab.update_traces(textposition='outside')
fig_diab.update_layout(showlegend=False, yaxis_tickformat='.1%')
fig_diab.show()

Encore une fois, le résultat est très cohérent : les personnes diabétiques ont un taux de crises cardiaques bien plus élevé. 

- Poids :

On regarde l'IMC (Indice de Masse Corporelle) des personnes appelées, qui est un indicateur du poids par rapport à la taille. 

In [14]:
# Mapping des catégories d'IMC pour la colonne _BMI5CAT
bmi_labels = {
    1: "Insuffisance pondérale",
    2: "Poids normal",
    3: "Surpoids",
    4: "Obésité"
}

# Filtrer les valeurs connues (1 à 4)
df_bmi = df_train[df_train['_BMI5CAT'].isin([1, 2, 3, 4])].copy()
df_bmi['Catégorie IMC'] = df_bmi['_BMI5CAT'].map(bmi_labels)

# Calcul du taux de crise cardiaque par catégorie d'IMC
bmi_group = df_bmi.groupby('Catégorie IMC')
total_by_bmi = bmi_group.size()
cardiac_by_bmi = bmi_group['TARGET'].sum()
taux_cardiac_bmi = (cardiac_by_bmi / total_by_bmi).reset_index()
taux_cardiac_bmi.columns = ['Catégorie IMC', 'Taux de crise cardiaque']

import plotly.express as px

fig_bmi = px.bar(
    taux_cardiac_bmi,
    x='Catégorie IMC',
    y='Taux de crise cardiaque',
    color='Catégorie IMC',
    color_discrete_sequence=plotly_colors,
    text=taux_cardiac_bmi['Taux de crise cardiaque'].apply(lambda x: f"{x:.2%}"),
    title="Taux de crise cardiaque selon la catégorie d'IMC"
)
fig_bmi.update_traces(textposition='outside')
fig_bmi.update_layout(showlegend=False, yaxis_tickformat='.1%')
fig_bmi.show()

: 